In [1]:
from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI

import getpass
import os
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import bs4
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [2]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
#OpenAI API 키 설정

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
langchain_api_key = os.environ.get("LANGCHAIN_API_KEY")
#Langchain API 키 설정

In [4]:
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0) 낮을수록 같은 질문에 같은 대답
    max_tokens=3000,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    #streaming=True,
    #callbacks=[StreamingStdOutCallbackHandler()], #스트리밍으로 답변 받기
)
# GPT LLM 생성

In [5]:
# PDF 파일 로드
loader = DirectoryLoader(".", glob="SkinLution_data/*.pdf", show_progress=True)

pdf_docs = loader.load()
print(f"문서의 수: {len(pdf_docs)}\n")

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:14<00:00,  1.84s/it]

문서의 수: 8



In [6]:
for i in range(len(pdf_docs)):
    print(pdf_docs[i].metadata)

{'source': 'SkinLution_data\\The_Effect_of_P.pdf'}
{'source': 'SkinLution_data\\고려엉겅퀴의_HPLC_패턴_.pdf'}
{'source': 'SkinLution_data\\목단피_에탄올_추출물의_B1.pdf'}
{'source': 'SkinLution_data\\세안화장품_사용에_따른_20.pdf'}
{'source': 'SkinLution_data\\아토피_피부의_임상연구__J.pdf'}
{'source': 'SkinLution_data\\울금의_항산화_및_미백효과.pdf'}
{'source': 'SkinLution_data\\제주도_토착_우뭇가사리의_항.pdf'}
{'source': 'SkinLution_data\\화장품에서_키토산_성분의_기.pdf'}


In [7]:
# TXT파일 로드
loader = DirectoryLoader(".", glob="SkinLution_data/*.txt", show_progress=True)

txt_docs = loader.load()
print(f"문서의 수: {len(txt_docs)}")

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 12.20it/s]

문서의 수: 24


In [8]:
for i in range(len(txt_docs)):
    print(txt_docs[i].metadata)

{'source': 'SkinLution_data\\3분 피부관리 꿀팁 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\눈가주름 눈밑주름 없애는 법 - 자글자글하면 꼭 보세요 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\다크서클 없애는 법 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\돈 안 들이고 여드름흉터를 치료할 수 있다고! 노스카나겔의 효과를 논문을 근거로 팩트체크합니다  노스카나겔 구매 전 필수 시청! - 피부과 전문의 왕정윤.txt'}
{'source': 'SkinLution_data\\돈 안들이고 피부 좋아지는 법 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\돈 안들이고 피부노화 막는 법 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\모공 작아지려면 어떻게 해야하나요 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\모공을 줄이는 3가지 원칙 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\서울대 피부과전문의가 말하는 피부좋아지는법  홈케어만으로 물광피부 완성! - 피부과 전문의 왕정윤.txt'}
{'source': 'SkinLution_data\\서울대 피부과전문의가 장담합니다! 비싼 화장품에 헛돈 쓰지 마시고, 이것만 바르면 피부가 새롭게 태어납니다  광고 전혀 없음! 가성비 보장! 약국연고추천! - 피부과 전문의 왕정윤.txt'}
{'source': 'SkinLution_data\\성인여드름 이것만 보시면 됩니다  여드름없애는방법 여드름관리법 - 피부과 전문의 왕정윤.txt'}
{'source': 'SkinLution_data\\스킨, 토너만으로 피부 좋아지는 방법 [피부과전문의 피부심].txt'}
{'source': 'SkinLution_data\\얼굴 주름 팽팽하게 쫙 펴

In [9]:
loader = CSVLoader(file_path='SkinLution_data/화장품데이터.csv', encoding='utf-8')
csv_docs = loader.load()

print(f"문서의 수: {len(csv_docs)}")

문서의 수: 120


In [10]:
# Split
pdf_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, #길이 500
    chunk_overlap=100, #겹치는 내용 100
    add_start_index=True #add_start_index=True초기 Document 내에서 각 분할 Document가 시작되는 문자 인덱스가 메타데이터 속성 "start_index"로 유지되도록 설정
)

pdf_splits = pdf_splitter.split_documents(pdf_docs)
print(len(pdf_splits))

txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, 
    chunk_overlap=50, 
    add_start_index=True #add_start_index=True초기 Document 내에서 각 분할 Document가 시작되는 문자 인덱스가 메타데이터 속성 "start_index"로 유지되도록 설정
)

txt_splits = txt_splitter.split_documents(txt_docs)
print(len(txt_splits))

460
461


In [11]:
#Store
# FAISS 또는 Chroma
# FAISS VectorStore를 생성
vectorstore1 = FAISS.from_documents(documents=pdf_splits, embedding=OpenAIEmbeddings())
vectorstore2 = FAISS.from_documents(documents=txt_splits, embedding=OpenAIEmbeddings())

cosmetic_vectorstore = FAISS.from_documents(documents=csv_docs, embedding=OpenAIEmbeddings())
vectorstore1.merge_from(vectorstore2) 

C:\Users\hoban\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
# k is the number of chunks to retrieve
retriever1 = vectorstore1.as_retriever(k=3)
retriever2 = cosmetic_vectorstore.as_retriever(k=50)

In [13]:
# 선택된 벡터스토어에 따라 검색 수행하는 함수
def is_cosmetic_query(user_message):
    # 간단한 키워드 기반 분류기 => 머신러닝 모델로 바꿀 수 있음(사용자의 질문이 화장품 추천을 바라는 것인지 아닌지로 레이블링)
    cosmetic_keywords = ["화장품", "제품", "스킨케어", "로션", "크림"]
    return any(keyword in user_message for keyword in cosmetic_keywords)
                         

# 적절한 vectorstore를 선택하는 로직
def select_vectorstore(user_message):
    if is_cosmetic_query(user_message):
        return retriever2
    else:
        return retriever1

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain

def perform_retrieval(user_message):
    selected_vectorstore = select_vectorstore(user_message)
    # 질의 변환 및 검색 수행
    query_transform_prompt = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="messages"),
            (
                "user",
                "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
            ),
        ]
    )

    question_answering_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Answer the user's question based on the below context:\n\n{context}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    query_transforming_retriever_chain = RunnableBranch(
        (
            lambda x: len(x.get("messages", [])) == 1,
            (lambda x: x["messages"][-1].content) | selected_vectorstore,
        ),
        query_transform_prompt 
        | llm 
        | StrOutputParser() 
        | selected_vectorstore,
    ).with_config(run_name="chat_retriever_chain")

    document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

    conversational_retrieval_chain = RunnablePassthrough.assign(
        context=query_transforming_retriever_chain,
    ).assign(
        answer=document_chain,
    )

    return conversational_retrieval_chain

In [16]:
demo_ephemeral_chat_history = ChatMessageHistory()

while True:
    user_message = input("질문할 내용을 입력하세요: ")
    if user_message == "-1":
        break
    demo_ephemeral_chat_history.add_user_message(user_message)
    retrieval_chain = perform_retrieval(user_message)
    response = retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
    
    # Add AI response to the chat history
    demo_ephemeral_chat_history.add_ai_message(response["answer"])
    
    # Extract the title from the first document in the context
    first_document_title = None
    if "context" in response and response["context"]:
        first_document = response["context"][0]
        first_document_title = first_document.page_content.split('\n')[0].strip()
    
    # Filter and print only AI messages
    ai_messages = [msg.content for msg in demo_ephemeral_chat_history.messages if isinstance(msg, AIMessage)]
    for msg in ai_messages:
        print(msg)
    
    # Print the title of the first document, if available
    if first_document_title:
        print(f"\n답변에 활용한 데이터: {first_document_title}\n")

질문할 내용을 입력하세요: 집에서 피부관리 하는 방법에 대해 알려줄래?
집에서 피부 관리하는 방법에 대해 알려드리겠습니다. 

1. 적절한 세안: 부드러운 클렌징 제품을 사용하여 피부를 깨끗하게 세정해주는 것이 중요합니다. 특히 메이크업을 하신 경우에는 이중 세안을 권장합니다.

2. 적절한 보습: 피부 타입에 맞는 보습제를 사용하여 피부를 촉촉하게 유지해주는 것이 중요합니다. 건성 피부일 경우에는 보습 크림을, 지성 피부일 경우에는 젤 형태의 보습제를 사용하는 것이 좋습니다.

3. 자외선 차단: 햇빛으로부터 피부를 보호하기 위해 자외선 차단제를 꾸준히 사용하는 것이 중요합니다. SPF 지수가 높은 제품을 선택하여 햇빛으로부터 피부를 지켜주세요.

4. 꾸준한 스킨케어: 일정한 주기로 필요한 스킨케어를 실시하여 피부 상태를 유지해주는 것이 중요합니다. 스킨토닉, 에센스, 세럼, 크림 순서대로 사용하여 피부에 영양을 공급해주세요.

이렇게 집에서 꾸준히 피부 관리를 하면 건강하고 빛나는 피부를 유지할 수 있을 것입니다. 부디 건강한 피부를 위해 노력해주세요.

Source Title: 서울대 피부과전문의가 말하는 피부좋아지는법 | 홈케어만으로 물광피부 완성!

질문할 내용을 입력하세요: -1


In [17]:
def skin_type_test():
    # 건성 피부 질문
    dry_questions = [
        "세안 후 피부가 땅기거나 건조함을 느끼나요?",
        "피부에 각질이 잘 생기나요?",
        "세안 후 보습제를 바로 발라야 편안함을 느끼나요?",
        "피부가 거칠거나 푸석푸석한 느낌이 드나요?",
        "추운 날씨나 건조한 환경에서 피부가 더 건조해지나요?",
        "피부에 피지나 유분이 거의 느껴지지 않나요?",
        "화장이 쉽게 들뜨고 갈라지는 편인가요?",
        "가렵거나 자극을 잘 받는 피부 상태인가요?"
    ]

    # 지성 피부 질문
    oily_questions = [
        "세안 후에도 피부에 기름기가 남아있나요?",
        "하루 동안 피부가 번들거리는 느낌이 드나요?",
        "모공이 크고 눈에 잘 띄나요?",
        "블랙헤드나 화이트헤드가 자주 생기나요?",
        "여드름이 자주 발생하나요?",
        "세안 후에도 금방 기름이 올라오는 편인가요?",
        "더운 날씨나 습한 환경에서 피부가 더 번들거리나요?",
        "화장이 쉽게 지워지고 번들거리는 편인가요?"
    ]

    dry_count = 0
    oily_count = 0

    print("건성 피부 질문에 '예' 또는 '아니오'로 답해주세요.")
    for question in dry_questions:
        answer = input(question + " (예/아니오): ").lower()
        if answer == "예":
            dry_count += 1

    print("\n지성 피부 질문에 '예' 또는 '아니오'로 답해주세요.")
    for question in oily_questions:
        answer = input(question + " (예/아니오): ").lower()
        if answer == "예":
            oily_count += 1

    if dry_count >= 5 and oily_count >= 5:
        print("\n복합성 피부입니다.")
    elif oily_count >= 5:
        print("\n지성 피부입니다.")
    elif dry_count >= 5:
        print("\n건성 피부입니다.")
    else:
        print("\n피부 타입을 정확히 판단할 수 없습니다. 추가적인 검사가 필요할 수 있습니다.")

# 피부 타입 검사 시작
skin_type_test()

건성 피부 질문에 '예' 또는 '아니오'로 답해주세요.


KeyboardInterrupt: Interrupted by user